In [ ]:
# Table 1A – AUROC & Δ-Mean
import json
import pandas as pd
from sklearn.metrics import roc_auc_score

# === Load JSON file ===
with open("/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withDetectors.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# === Supported detectors and their probability fields ===
detectors = {
    "originality": lambda d: d.get("confidence", {}).get("AI", None),
    "pengram": lambda d: d.get("ai_likelihood", None),
    "gptzero": lambda d: d.get("average_generated_prob", None),
    "roberta-base-detector": lambda d: d if isinstance(d, (float, int)) else None
}

# === Collect scores ===
records = []

for entry in data:
    # Human text
    hv = entry.get("human_verdict", {})
    for det, extract in detectors.items():
        if det in hv:
            prob = extract(hv[det])
            if prob is not None:
                records.append({
                    "detector": det,
                    "source": "human",
                    "score": prob,
                    "label": 0
                })

    # AI-generated text
    for model, verdicts in entry.get("ai_verdicts", {}).items():
        for det, extract in detectors.items():
            if det in verdicts:
                prob = extract(verdicts[det])
                if prob is not None:
                    records.append({
                        "detector": det,
                        "source": f"ai_{model}",
                        "score": prob,
                        "label": 1
                    })

# === Convert to DataFrame ===
df = pd.DataFrame(records)

# === Compute AUROC & Δ-Mean for each detector ===
summary = []

for det in df["detector"].unique():
    sub = df[df["detector"] == det]
    ai_scores = sub[sub["label"] == 1]["score"]
    human_scores = sub[sub["label"] == 0]["score"]
    all_labels = sub["label"]
    all_scores = sub["score"]

    try:
        auroc = roc_auc_score(all_labels, all_scores)
    except:
        auroc = None

    delta_mean = ai_scores.mean() - human_scores.mean()

    summary.append({
        "detector": det,
        "AUROC": round(auroc, 4) if auroc is not None else "N/A",
        "Δ-Mean": round(delta_mean, 4)
    })

# === Final Table 1A Output ===
summary_df = pd.DataFrame(summary).sort_values(by="AUROC", ascending=False)
# print(summary_df.to_string(index=False))
summary_df.to_csv("/content/drive/MyDrive/detection/pre2020_human_text_by_genre/result/Table1_overall.csv")
summary_df

,detector,AUROC,Δ-Mean
0,originality,1.0000,0.9959
1,pengram,1.0000,0.9911
2,gptzero,0.9933,0.9866
3,roberta-base-detector,0.2745,-0.3351


In [ ]:
# Table 1 By genre / domain

import json
import pandas as pd
from sklearn.metrics import roc_auc_score

# === Load JSON ===
with open("/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withDetectors.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# === Detector: field extractors ===
detectors = {
    "originality": lambda d: d.get("confidence", {}).get("AI", None),
    "pengram": lambda d: d.get("ai_likelihood", None),
    "gptzero": lambda d: d.get("average_generated_prob", None),
    "roberta-base-detector": lambda d: d if isinstance(d, (float, int)) else None
}

# === Collect records with genre ===
records = []

for entry in data:
    genre = entry.get("genre", "unknown")
    hv = entry.get("human_verdict", {})
    for det, extract in detectors.items():
        if det in hv:
            prob = extract(hv[det])
            if prob is not None:
                records.append({
                    "genre": genre,
                    "detector": det,
                    "score": prob,
                    "label": 0
                })

    for model, verdicts in entry.get("ai_verdicts", {}).items():
        for det, extract in detectors.items():
            if det in verdicts:
                prob = extract(verdicts[det])
                if prob is not None:
                    records.append({
                        "genre": genre,
                        "detector": det,
                        "score": prob,
                        "label": 1
                    })

# === Convert to DataFrame
df = pd.DataFrame(records)

# === Compute AUROC & Δ-Mean by genre x detector
summary = []

for (genre, det), sub in df.groupby(["genre", "detector"]):
    ai_scores = sub[sub["label"] == 1]["score"]
    human_scores = sub[sub["label"] == 0]["score"]
    all_labels = sub["label"]
    all_scores = sub["score"]

    try:
        auroc = roc_auc_score(all_labels, all_scores)
    except:
        auroc = None

    delta_mean = ai_scores.mean() - human_scores.mean()

    summary.append({
        "genre": genre,
        "detector": det,
        "AUROC": round(auroc, 4) if auroc is not None else "N/A",
        "Δ-Mean": round(delta_mean, 4),
        "n_ai": len(ai_scores),
        "n_human": len(human_scores)
    })

# === Format with rank per genre
summary_df = pd.DataFrame(summary)

# === Global sort by AUROC across all (genre, detector) combinations
# summary_df["Rank"] = summary_df["AUROC"].rank(ascending=False, method="min").astype(int)

# Sort by AUROC descending
summary_df = summary_df.sort_values("AUROC", ascending=False).reset_index(drop=True)
summary_df

,genre,detector,AUROC,Δ-Mean,n_ai,n_human
0,amazon review,pengram,1.0000,0.9836,200,200
1,blog,originality,1.0000,0.9928,139,142
2,blog,pengram,1.0000,0.9932,200,200
3,resume,originality,1.0000,0.9994,200,200
4,novel,originality,1.0000,0.9995,996,999
5,novel,pengram,1.0000,1.0000,999,1000
6,news,pengram,1.0000,0.9959,300,300
7,news,originality,1.0000,0.9935,291,292
8,restaurant review,originality,1.0000,0.9803,138,140
9,resume,pengram,1.0000,1.0000,200,200


In [ ]:
summary_df.to_csv("/content/drive/MyDrive/detection/pre2020_human_text_by_genre/result/Table1_ByGenre.csv")

# Reverse-engineer the threshold based on the FP rate

In [ ]:

# Reverse-engineer the threshold based on the false positive (FP) rate (pengram)
import json
import numpy as np

def compute_pengram_threshold(data, fpr_target=0.1): #change the ideal fp rate here
    human_scores = []
    for item in data:
        try:
            score = item['human_verdict']['pengram']['ai_likelihood']
            human_scores.append(score)
        except KeyError:
            continue

    if not human_scores:
        raise ValueError("❌ error")

    human_scores_sorted = np.sort(human_scores)
    cutoff_index = int(np.floor((1 - fpr_target) * len(human_scores_sorted))) - 1
    threshold = human_scores_sorted[cutoff_index]

    print(f"✅ pengram threshold is：{threshold:.8f}")
    return threshold

with open('/content/drive/MyDrive/detection/pre2020_human_text_by_genre/all_json_files/merged_corpus_withDetectors_defaultProb.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

compute_pengram_threshold(dataset)


✅ 1% 假阳性率对应的 pengram 阈值为：0.00001593


np.float64(1.592768563164605e-05)

In [ ]:
# Reverse-engineer the threshold based on the FP rate (originality)
import json
import numpy as np

def compute_pengram_threshold(data, fpr_target=0.1):

    human_scores = []
    for item in data:
        try:
            score = item['human_verdict']['originality']['confidence']['AI']
            human_scores.append(score)
        except KeyError:
            continue

    if not human_scores:
        raise ValueError("❌ error")

    human_scores_sorted = np.sort(human_scores)
    cutoff_index = int(np.floor((1 - fpr_target) * len(human_scores_sorted))) - 1
    threshold = human_scores_sorted[cutoff_index]

    print(f" originality's threshold is: {threshold:.8f}")
    return threshold

with open('/content/drive/MyDrive/detection/pre2020_human_text_by_genre/all_json_files/merged_corpus_withDetectors_defaultProb.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

compute_pengram_threshold(dataset)


 originality's threshold is: 0.00120000


np.float64(0.0012)

In [ ]:
# Reverse-engineer the threshold based on the FP rate（GPTzero）
import json
import numpy as np

def compute_pengram_threshold(data, fpr_target=0.1):

    human_scores = []
    for item in data:
        try:
            score = item['human_verdict']['gptzero']["average_generated_prob"]
            human_scores.append(score)
        except KeyError:
            continue

    if not human_scores:
        raise ValueError("❌ 没有有效的 GPTzero 分数可用于计算")

    human_scores_sorted = np.sort(human_scores)
    cutoff_index = int(np.floor((1 - fpr_target) * len(human_scores_sorted))) - 1
    threshold = human_scores_sorted[cutoff_index]

    print(f"GPTzero's threshold is：{threshold:.8f}")
    return threshold

with open('/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withDetectors.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

compute_pengram_threshold(dataset)


✅ 1% 假阳性率对应的 GPTzero 阈值为：0.00000000


np.int64(0)

In [ ]:
# Reverse-engineer the threshold based on the FP rate (roberta-base-detector)

import json
import numpy as np

def compute_roberta_threshold(data, fpr_target=0.1):
    human_scores = []

    for item in data:
        try:
            score = item['human_verdict']['roberta-base-detector']
            human_scores.append(score)
        except KeyError:
            continue

    if not human_scores:
        raise ValueError("❌ 没有有效的 roberta-base-detector 分数可用于计算")

    human_scores_sorted = np.sort(human_scores)
    cutoff_index = int(np.floor((1 - fpr_target) * len(human_scores_sorted))) - 1
    threshold = human_scores_sorted[cutoff_index]

    print(f"✅ 为控制假阳性率在 {fpr_target*100:.2f}% 内，roberta-base-detector 的阈值应为：{threshold:.6f}")
    return threshold

with open('/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withDetectors.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

compute_roberta_threshold(dataset)


✅ 为控制假阳性率在 10.00% 内，roberta-base-detector 的阈值应为：0.999826


np.float64(0.9998257756233215)

# Following
Title: Error Analysis of AI Text Detectors by Genre

Description:
This script evaluates the performance of multiple AI text detection models (e.g., originality, pengram, GPTZero, RoBERTa-based)
on both human-written and AI-generated texts. It calculates Type I and Type II error rates, as well as precision and recall,
grouped by text genre and detector.

It uses Wilson confidence intervals and binomial tests for statistical estimation, and supports JSON input formatted with
"human_verdict" and "ai_verdicts" fields.

Output includes:
- False positive (Type I) and false negative (Type II) rates
- Precision and recall with confidence intervals
"""


In [1]:


import json
import pandas as pd
import numpy as np
import scipy.stats as st

# Load the dataset
input_path = "/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withThreshold_FP0.1.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Prepare the record list
records = []
for item in data:
    # Get genre info, use "unknown" if missing
    genre = item.get("genre", "unknown")

    for detector_name in ["originality", "pengram", "gptzero", "roberta-base-detector"]:
        # Process human text verdicts
        if "human_verdict" in item and detector_name in item["human_verdict"]:
            verdict = item["human_verdict"].get(detector_name)
            if verdict is not None:
                records.append({
                    "genre": genre,
                    "detector": detector_name,
                    "is_human": True,
                    "verdict": verdict
                })

        # Process AI text verdicts
        if "ai_verdicts" in item and detector_name in item.get("ai_verdicts", {}):
            verdict = item["ai_verdicts"].get(detector_name)
            if verdict is not None:
                records.append({
                    "genre": genre,
                    "detector": detector_name,
                    "is_human": False,
                    "verdict": verdict
                })

df = pd.DataFrame(records)

# Compute Wilson confidence interval
def wilson_ci(k, n, alpha=0.05):
    if n == 0:
        return (0, 0)
    z = st.norm.ppf(1 - alpha/2)
    phat = k / n
    denom = 1 + z**2/n
    center = phat + z**2 / (2 * n)
    pm = z * ((phat * (1 - phat) / n + z**2 / (4 * n**2)) ** 0.5)
    return ((center - pm) / denom, (center + pm) / denom)

# Evaluate error rates, precision, and recall by genre and detector
results = []
metrics_results = []

for (genre, detector), group in df.groupby(["genre", "detector"]):
    human = group[group["is_human"]]
    ai = group[~group["is_human"]]

    # Compute confusion matrix components
    total_human = human["verdict"].notna().sum()
    total_ai = ai["verdict"].notna().sum()

    fp = (human["verdict"] == 1).sum()  # False Positives: human texts incorrectly flagged as AI
    tn = (human["verdict"] == 0).sum()  # True Negatives: human texts correctly flagged as human
    tp = (ai["verdict"] == 1).sum()     # True Positives: AI texts correctly flagged as AI
    fn = (ai["verdict"] == 0).sum()     # False Negatives: AI texts incorrectly flagged as human

    # Type I error (False Positive Rate)
    type1_rate = fp / total_human if total_human else np.nan
    type1_ci = wilson_ci(fp, total_human)
    type1_p = st.binomtest(fp, total_human, p=0.5, alternative='greater').pvalue if total_human else np.nan

    # Type II error (False Negative Rate)
    type2_rate = fn / total_ai if total_ai else np.nan
    type2_ci = wilson_ci(fn, total_ai)
    type2_p = st.binomtest(fn, total_ai, p=0.5, alternative='greater').pvalue if total_ai else np.nan

    # Calculate Precision and Recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else np.nan
    precision_ci = wilson_ci(tp, tp + fp) if (tp + fp) > 0 else (np.nan, np.nan)

    recall = tp / (tp + fn) if (tp + fn) > 0 else np.nan
    recall_ci = wilson_ci(tp, tp + fn) if (tp + fn) > 0 else (np.nan, np.nan)

    # Append error rate results
    results.append({
        "Genre": genre,
        "Detector": detector,
        "Error Type": "Type I (False Positive)",
        "Count": fp,
        "Total": total_human,
        "Error Rate": type1_rate,
        "CI Lower": type1_ci[0],
        "CI Upper": type1_ci[1],
        "p-value": type1_p
    })
    results.append({
        "Genre": genre,
        "Detector": detector,
        "Error Type": "Type II (False Negative)",
        "Count": fn,
        "Total": total_ai,
        "Error Rate": type2_rate,
        "CI Lower": type2_ci[0],
        "CI Upper": type2_ci[1],
        "p-value": type2_p
    })

    # Append precision and recall results
    metrics_results.append({
        "Genre": genre,
        "Detector": detector,
        "Metric": "Precision",
        "Value": precision,
        "CI Lower": precision_ci[0],
        "CI Upper": precision_ci[1],
        "True Positives": tp,
        "False Positives": fp,
        "Total Detected as AI": tp + fp
    })
    metrics_results.append({
        "Genre": genre,
        "Detector": detector,
        "Metric": "Recall",
        "Value": recall,
        "CI Lower": recall_ci[0],
        "CI Upper": recall_ci[1],
        "True Positives": tp,
        "False Negatives": fn,
        "Total AI": tp + fn
    })

results_df = pd.DataFrame(results)
metrics_df = pd.DataFrame(metrics_results)




FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withThreshold_FP0.1.json'

In [ ]:
metrics_df

In [ ]:
results_df